In [1]:
import pandas as pd
import datetime

This notebook is preliminary ETL for data from the weather API. <br>
We work with data for Union Square in San Francisco; (37.7879, -122.4079) for API.  <br>
<b> Beware, returned data says has coordinates (38, -122.75) which is 30 miles north in Point Reyes! </b> <br>
We pulled data from 2000-01-01 to 2020-12-31. <br>
We took temperature, humidity, rainfal, snowfal, cloudcover, windspeed, and wind-direction hourly. <br>
<hr> 
We will also take max temp, min temp, rain, snow, and percip-hours daily. <br>
These will need separate and different dataframe processing. <br>
We are taking in data via downloaded csv, and we need separate daily/hourly csv files because different column headers, numbers of columns.<br>
<hr>
We chose ISO format for date/time; and USian (not-metric) units for the rest.<br>


This run is with "GMT +0" time zone; in the future, we will need to specify the timezone appropriate to the location.<br>
API allows "GMT +0" time zone for hourly data, but not for daily data - says "error, time zone must be specified" -- except does time zone even matter for dailies? I think not for us, except for reproducibility.<br>
If we're stuck with something other than "GMT +0", we could get strange behaviour whenif daylight savings time kicks in; can check for that by requesting sunrise time data.

## Hourly data: load, clean, pivot for joining with daily.

In [2]:
# Load the dataset.
# rename columns because provided column headers contain non-ascii characters
file_path ="hourly.csv"
hourly_columns = ['time',
           'temperature_2m_degF', 
           'relativehumidity_2m_perc', 
           'rain_inch',
           'snowfall_cm', 
           'cloudcover_perc', 
           'windspeed_10m_mph',
           'winddirection_10m_deg']

raw_hrly_df1 = pd.read_csv(file_path, skiprows=4, names=hourly_columns)

# When wind speed is 0, wind direction is NaN. Otherwise, wind directions varies from 1 to 360.
# We replace NaNs with 0s to avoid errors; and losing no data as 0 never appears in the original.
raw_hrly_df1["winddirection_10m_deg"].fillna(0, inplace=True)

# We convert the provided ISO string 'time' into
#    a 'pure_date' in python datetime format for merging with daily data; and
#    an 'hour' integer, for pivoting.

raw_hrly_df1["pure_date"] = raw_hrly_df1['time'].map(lambda x: 
                                                     datetime.datetime.fromisoformat(x[0:10]))
raw_hrly_df1["hour"] = raw_hrly_df1['time'].map(lambda x: datetime.datetime.fromisoformat(x).hour)

# Now we pivot the dataframe on the "hour" variable: for each value of 'pure_date',
#  - The raw dataframe has 24 rows (one for each hour) with: 7 weather columns; and
#   3 date/time columns: original string 'time', datetime 'pure_date', integer 'hour; and
#    mostly meaningless sequential index.
#   The clean dataframe has 1 row with: index 'pure_date'; and 7*24=168 weather-at-hour columns. 

clean_hrly_df1 = raw_hrly_df1.pivot(index = 'pure_date',
                                    columns = 'hour', values = hourly_columns[1:])
clean_hrly_df1.head()

# The two percentage columns that used to be int64 got converted to float64 during pivoting.

temperature_2m_degF                                            \
hour                        0     1     2     3     4     5     6     7    
pure_date                                                                  
2000-01-01                51.6  51.3  51.4  48.1  47.8  47.8  46.7  45.6   
2000-01-02                52.2  51.8  51.2  47.8  47.8  47.6  47.5  47.2   
2000-01-03                52.2  52.1  51.8  48.1  48.3  48.0  47.9  47.9   
2000-01-04                52.9  52.0  50.9  47.6  47.9  49.4  49.2  47.3   
2000-01-05                53.0  53.2  53.2  49.8  49.2  49.4  50.0  50.0   

                        ... winddirection_10m_deg                              \
hour          8     9   ...                    14     15     16     17     18   
pure_date               ...                                                     
2000-01-01  45.0  43.1  ...                   3.0    7.0  350.0  333.0  315.0   
2000-01-02  46.8  45.4  ...                 326.0  327.0  329.0  327.0  323.0   
2000-01-03  47.7  46.9  ...                 352.0    8.0   27.0   29.0  339.0   
2000-01-04  47.3  45.9  ...                 280.0  277.0  291.0  261.0  180.0   
2000-01-05  49.7  49.2  ...                 342.0  343.0  335.0  324.0  318.0   

                                               
hour           19     20     21     22     23  
pure_date                                      
2000-01-01  306.0  307.0  308.0  296.0  297.0  
2000-01-02  317.0  309.0  306.0  303.0  302.0  
2000-01-03  315.0  252.0  267.0  275.0  283.0  
2000-01-04  162.0  151.0  161.0  188.0  195.0  
2000-01-05  318.0  319.0  323.0  314.0  310.0  

[5 rows x 168 columns]

### Load daily data

In [5]:
# Load the dataset.
# rename columns because provided column headers contain non-ascii characters
file_path ="daily.csv"
daily_columns = ['time',
           'temperature_2m_degF_max',
           'temperature_2m_degF_min',  
           'rain_inch',
           'snowfall_cm',
           'precipitation_hours'
          ]

raw_daily_df1 = pd.read_csv(file_path, skiprows=4, names=daily_columns)

# convert ISO date into datetime
raw_daily_df1["date"] = raw_daily_df1['time'].map(lambda x: datetime.datetime.fromisoformat(x))
raw_daily_df1.drop('time', axis=1, inplace=True)

raw_daily_df1.head()

,temperature_2m_degF_max,temperature_2m_degF_min,rain_inch,snowfall_cm,precipitation_hours,date
0,52.2,43.1,0.00,0.0,0.0,2000-01-01
1,52.3,45.1,0.00,0.0,0.0,2000-01-02
2,52.9,42.8,0.00,0.0,0.0,2000-01-03
3,53.2,44.2,0.02,0.0,2.0,2000-01-04
4,56.8,45.8,0.00,0.0,0.0,2000-01-05


### Join!

In [6]:
# warning says, flatten column names or else join will throw errors in future versions
clean_hrly_df1.columns = clean_hrly_df1.columns.to_flat_index()
awesome_df = raw_daily_df1.join(clean_hrly_df1, on = 'date' )
awesome_df.head()

,temperature_2m_degF_max,temperature_2m_degF_min,rain_inch,snowfall_cm,precipitation_hours,date,"(temperature_2m_degF, 0)","(temperature_2m_degF, 1)","(temperature_2m_degF, 2)","(temperature_2m_degF, 3)",...,"(winddirection_10m_deg, 14)","(winddirection_10m_deg, 15)","(winddirection_10m_deg, 16)","(winddirection_10m_deg, 17)","(winddirection_10m_deg, 18)","(winddirection_10m_deg, 19)","(winddirection_10m_deg, 20)","(winddirection_10m_deg, 21)","(winddirection_10m_deg, 22)","(winddirection_10m_deg, 23)"
0,52.2,43.1,0.00,0.0,0.0,2000-01-01,51.6,51.3,51.4,48.1,...,3.0,7.0,350.0,333.0,315.0,306.0,307.0,308.0,296.0,297.0
1,52.3,45.1,0.00,0.0,0.0,2000-01-02,52.2,51.8,51.2,47.8,...,326.0,327.0,329.0,327.0,323.0,317.0,309.0,306.0,303.0,302.0
2,52.9,42.8,0.00,0.0,0.0,2000-01-03,52.2,52.1,51.8,48.1,...,352.0,8.0,27.0,29.0,339.0,315.0,252.0,267.0,275.0,283.0
3,53.2,44.2,0.02,0.0,2.0,2000-01-04,52.9,52.0,50.9,47.6,...,280.0,277.0,291.0,261.0,180.0,162.0,151.0,161.0,188.0,195.0
4,56.8,45.8,0.00,0.0,0.0,2000-01-05,53.0,53.2,53.2,49.8,...,342.0,343.0,335.0,324.0,318.0,318.0,319.0,323.0,314.0,310.0


In [7]:
# Parse date into numbers: year, month, and day-of-the-year.
# The 'month' field is for the month-label machine learning. 
# The day_of_year field is for the season-clusters machine learning. 
# The 'year' field is for maybe pivoting on, for the season-clusters machine learning. 
# The day_of_year field runs from 1 to 365 (366 for leap years): February 3rd would be 34. 
# We keep the whole date formatted as datetime for time intervals like 2011-12-28 to 2012-01-17.

awesome_df['year'] = awesome_df["date"].map(lambda x: x.year)
awesome_df['month'] = awesome_df["date"].map(lambda x: x.month)

# the next tw lines of code are stolen from
# https://www.w3resource.com/python-exercises/date-time-exercise/python-date-time-exercise-11.php
# which says nothing about licenses
def day_of_year(x):
    return (x - datetime.datetime(x.year, 1, 1)).days + 1

awesome_df['day_of_year'] = awesome_df["date"].map(lambda x: day_of_year(x))

In [9]:
# save the result for other people's use 
awesome_df.to_csv('awesome.csv')

In [25]:
# this function reads in data from the first two filename inputs,
# performs ETL, and writes the result to the third filename input
# NO ERROR HANDLING

def etl_step1(hourly_input_file, daily_input_file, output_file):
    # Load the hourly dataset.
    # rename columns because provided column headers contain non-ascii characters
    hourly_columns = ['time', 'temperature_2m_degF', 'relativehumidity_2m_perc',
                      'rain_inch', 'snowfall_cm', 'cloudcover_perc',
                      'windspeed_10m_mph', 'winddirection_10m_deg']
    raw_hrly_df1 = pd.read_csv(hourly_input_file, skiprows=4, names=hourly_columns)

    # When wind speed is 0, wind direction is NaN. Otherwise, wind directions varies from 1 to 360.
    # We replace NaNs with 0s to avoid errors; and losing no data as 0 never appears in the original.
    raw_hrly_df1["winddirection_10m_deg"].fillna(0, inplace=True)

    # We convert the provided ISO string 'time' into
    #    a 'pure_date' in python datetime format for merging with daily data; and
    #    an 'hour' integer, for pivoting.
    raw_hrly_df1["pure_date"] = raw_hrly_df1['time'].map(lambda x: 
                                                     datetime.datetime.fromisoformat(x[0:10]))
    raw_hrly_df1["hour"] = raw_hrly_df1['time'].map(lambda x: datetime.datetime.fromisoformat(x).hour)

    # Now we pivot the dataframe on the "hour" variable: for each value of 'pure_date',
    #  - The raw dataframe has 24 rows (one for each hour) with: 7 weather columns; and
    #   3 date/time columns: original string 'time', datetime 'pure_date', integer 'hour; and
    #    mostly meaningless sequential index.
    #   The clean dataframe has 1 row with: index 'pure_date'; and 7*24=168 weather-at-hour columns. 
    clean_hrly_df1 = raw_hrly_df1.pivot(index = 'pure_date',
                                    columns = 'hour', values = hourly_columns[1:])
    # The two percentage columns that used to be int64 got converted to float64 during pivoting.

    # Load the daily dataset.
    # rename columns because provided column headers contain non-ascii characters
    daily_columns = ['time', 'temperature_2m_degF_max','temperature_2m_degF_min', 
                     'rain_inch', 'snowfall_cm', 'precipitation_hours']
    raw_daily_df1 = pd.read_csv(daily_input_file, skiprows=4, names=daily_columns)
    # convert ISO date into datetime
    raw_daily_df1["date"] = raw_daily_df1['time'].map(lambda x: datetime.datetime.fromisoformat(x))
    raw_daily_df1.drop('time', axis=1, inplace=True)

    # join dataframes
    # warning says, flatten column names or else join will throw errors in future versions
    clean_hrly_df1.columns = clean_hrly_df1.columns.to_flat_index()
    awesome_df = raw_daily_df1.join(clean_hrly_df1, on = 'date' )
    
    # Parse date into numbers: year, month, and day-of-the-year.
    # The 'month' field is for the month-label machine learning. 
    # The day_of_year field is for the season-clusters machine learning. 
    # The 'year' field is for maybe pivoting on, for the season-clusters machine learning. 
    # The day_of_year field runs from 1 to 365 (366 for leap years): February 3rd would be 34. 
    # We keep the whole date formatted as datetime for time intervals like 2011-12-28 to 2012-01-17.
    awesome_df['year'] = awesome_df["date"].map(lambda x: x.year)
    awesome_df['month'] = awesome_df["date"].map(lambda x: x.month)
    # the next tw lines of code are stolen from
    # https://www.w3resource.com/python-exercises/date-time-exercise/python-date-time-exercise-11.php
    # which says nothing about licenses
    def day_of_year(x):
        return (x - datetime.datetime(x.year, 1, 1)).days + 1
    awesome_df['day_of_year'] = awesome_df["date"].map(lambda x: day_of_year(x))
    
    # finally, write out the result
    awesome_df.to_csv(output_file)

In [33]:
etl_step1('chi_hourly.csv', 'chi_daily.csv', 'chi_clean.csv')